In [16]:
import cv2 as cv
import torch
import matplotlib.pyplot as plt

# Energy calculation function using PyTorch
def energy(image):
    image = torch.tensor(image, dtype=torch.float32, device='cuda')
    if image.ndim == 3:
        energy_map = torch.zeros(image.shape[:2], device='cuda')
        for channel in range(3):
            sobel_x = torch.abs(torch.nn.functional.conv2d(
                image[:, :, channel].unsqueeze(0).unsqueeze(0),
                torch.tensor([[[[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]]]], device='cuda', dtype=torch.float32),
                padding=1
            ).squeeze())
            sobel_y = torch.abs(torch.nn.functional.conv2d(
                image[:, :, channel].unsqueeze(0).unsqueeze(0),
                torch.tensor([[[[-1, -2, -1], [0, 0, 0], [1, 2, 1]]]], device='cuda', dtype=torch.float32),
                padding=1
            ).squeeze())
            energy_map += torch.sqrt(sobel_x**2 + sobel_y**2)
    else:
        sobel_x = torch.abs(torch.nn.functional.conv2d(
            image.unsqueeze(0).unsqueeze(0),
            torch.tensor([[[[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]]]], device='cuda', dtype=torch.float32),
            padding=1
        ).squeeze())
        sobel_y = torch.abs(torch.nn.functional.conv2d(
            image.unsqueeze(0).unsqueeze(0),
            torch.tensor([[[[-1, -2, -1], [0, 0, 0], [1, 2, 1]]]], device='cuda', dtype=torch.float32),
            padding=1
        ).squeeze())
        energy_map = torch.sqrt(sobel_x**2 + sobel_y**2)
    return energy_map

# Remove a vertical seam using PyTorch
def remove_v_seam(image, indexes):
    mask = torch.ones(image.shape[:2], dtype=torch.bool, device='cuda')
    mask[torch.arange(image.shape[0]), indexes] = False
    return image[mask].reshape(image.shape[0], image.shape[1] - 1, image.shape[2])

# Remove a horizontal seam using PyTorch
def remove_h_seam(image, indexes):
    mask = torch.ones(image.shape[:2], dtype=torch.bool, device='cuda')
    mask[indexes, torch.arange(image.shape[1])] = False
    return image.permute(1, 0, 2)[mask.T].reshape(image.shape[1], image.shape[0] - 1, image.shape[2]).permute(1, 0, 2)

# Parallel seam finding using PyTorch
def parallel_seam_find(image, axis='v'):
    energy_matrix = energy(image)
    if axis == 'h':
        energy_matrix = energy_matrix.T

    rows, cols = energy_matrix.shape
    cost = energy_matrix.clone()
    backtrack = torch.zeros((rows, cols), dtype=torch.int32, device='cuda')

    for i in range(1, rows):
        left = torch.cat((cost[i - 1, 1:], torch.tensor([float('inf')], device='cuda')))
        right = torch.cat((torch.tensor([float('inf')], device='cuda'), cost[i - 1, :-1]))
        min_cost, min_index = torch.min(torch.stack([left, cost[i - 1], right]), dim=0)
        cost[i] += min_cost
        backtrack[i] = min_index - 1

    seam = torch.zeros(rows, dtype=torch.int32, device='cuda')
    seam[-1] = torch.argmin(cost[-1])
    for i in range(rows - 2, -1, -1):
        seam[i] = torch.clamp(seam[i + 1] + backtrack[i + 1, seam[i + 1]], min=0, max=cols - 1)

    return seam

# Highlight removed seams on the original image
def highlight_seams(image, seams, axis='v'):
    if axis == 'h':
        image[seams.long(), torch.arange(image.shape[1], device='cuda').long()] = torch.tensor([0, 0, 255], dtype=torch.float32, device='cuda')
    else:
        image[torch.arange(image.shape[0], device='cuda').long(), seams.long()] = torch.tensor([0, 0, 255], dtype=torch.float32, device='cuda')
    return image


In [18]:
import cv2 as cv
import numpy as np
import torch
import matplotlib.pyplot as plt
from PIL import Image
from google.colab import files

def upload_image():
    uploaded = files.upload()
    image_path = list(uploaded.keys())[0]  # Get the uploaded file name
    return image_path

def load_image(image_path):
    image = Image.open(image_path)
    image = image.convert('RGB')  # Ensure it's in RGB format
    return np.array(image)

def preprocess_image(image):
    return torch.tensor(image, dtype=torch.float32, device='cuda') / 255.0

def postprocess_image(image):
    return (image.cpu().numpy() * 255).astype(np.uint8)

def test_seam_carving(image_path):
    image = load_image(image_path)
    image_torch = preprocess_image(image)

    # Find and remove a vertical seam
    v_seam = parallel_seam_find(image_torch, axis='v')
    image_torch = remove_v_seam(image_torch, v_seam)

    # Find and remove a horizontal seam
    h_seam = parallel_seam_find(image_torch, axis='h')
    image_torch = remove_h_seam(image_torch, h_seam)

    # Highlight seams on the original image
    highlighted = highlight_seams(preprocess_image(image), v_seam, axis='v')
    highlighted = highlight_seams(highlighted, h_seam, axis='h')

    # Convert back to NumPy for visualization
    processed_image = postprocess_image(image_torch)
    highlighted_image = postprocess_image(highlighted)

    # Display images
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
    ax[0].imshow(image)
    ax[0].set_title("Original Image")
    ax[0].axis("off")

    ax[1].imshow(highlighted_image)
    ax[1].set_title("Highlighted Seams")
    ax[1].axis("off")

    ax[2].imshow(processed_image)
    ax[2].set_title("Image After Seam Removal")
    ax[2].axis("off")

    plt.show()

test_seam_carving("/content/sheep-on-a-meadow.jpg")


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [9]:
import cv2 as cv
import numpy as np
from imageio import imread, imwrite
import matplotlib.pyplot as plt

# Energy calculation using Sobel filters
def energy(image):
    gray = cv.cvtColor(image, cv.COLOR_RGB2GRAY).astype(np.float32)
    sobel_x = cv.Sobel(gray, cv.CV_64F, 1, 0, ksize=3)
    sobel_y = cv.Sobel(gray, cv.CV_64F, 0, 1, ksize=3)
    return np.abs(sobel_x) + np.abs(sobel_y)

# Find the minimum seam using dynamic programming
def find_seam(energy_map):
    rows, cols = energy_map.shape
    M = energy_map.copy()
    backtrack = np.zeros_like(M, dtype=np.int32)
    for i in range(1, rows):
        for j in range(cols):
            left = M[i-1, j-1] if j > 0 else float('inf')
            up = M[i-1, j]
            right = M[i-1, j+1] if j < cols-1 else float('inf')
            min_energy = min(left, up, right)
            backtrack[i, j] = j - 1 if min_energy == left else j + 1 if min_energy == right else j
            M[i, j] += min_energy

    seam = []
    j = np.argmin(M[-1])
    for i in reversed(range(rows)):
        seam.append((i, j))
        j = backtrack[i, j]
    return seam

# Remove a seam and highlight it
def remove_seam(img, seam):
    rows, cols, _ = img.shape
    mask = np.ones((rows, cols), dtype=np.bool_)
    for i, j in seam:
        mask[i, j] = False
    new_img = img[mask].reshape((rows, cols - 1, 3))
    return new_img

# Highlight seam on the image
def highlight_seam(img, seam):
    img_marked = img.copy()
    for i, j in seam:
        img_marked[i, j] = [255, 0, 0]  # Red color for seam
    return img_marked

# Seam carving process
def crop_c(img, scale_c):
    new_c = int(scale_c * img.shape[1])
    img_marked = img.copy()
    for _ in range(img.shape[1] - new_c):
        seam = find_seam(energy(img))
        img_marked = highlight_seam(img_marked, seam)
        img = remove_seam(img, seam)
    return img, img_marked

# Function to run in a notebook
def run_seam_carving(image_path, scale=0.9):
    img = imread(image_path)
    resized_img, marked_img = crop_c(img, scale)

    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    ax[0].imshow(marked_img)
    ax[0].set_title("Highlighted Seams")
    ax[0].axis("off")

    ax[1].imshow(resized_img)
    ax[1].set_title("Resized Image")
    ax[1].axis("off")

    plt.show()



In [11]:
run_seam_carving("/content/sheep-on-a-meadow.jpg",0.8)

<ipython-input-9-cf343da14ddb>:62: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = imread(image_path)


KeyboardInterrupt: 